In [ ]:
import random
import os
import csv

import h5py
from glob import glob

import cv2
import numpy as np 
import matplotlib.pyplot as plt
from PIL import Image, ImageFont, ImageDraw 

from sklearn.mixture import GaussianMixture

In [ ]:
path_in = "../../IN/H5PY/dataset_h5py/"
files = glob(path_in + "*.h5py")

In [ ]:
font = ImageFont.truetype(r"iosevka-regular.ttf", 20)

def draw_annotation(X, coord):
    X_draw = Image.fromarray(X)
    X_drawable = ImageDraw.Draw(X_draw)
    for point in coord:
        X_drawable.text((point[0]-20, point[1]-20,), str(point[3]), fill=point[2], font=font)
    return np.array(X_draw)

In [ ]:
treatments_out = {}

In [ ]:
thresh = 245/255

for p in files:
   name = os.path.splitext(os.path.basename(p))[0] 
   treatments_out[name] = {}
   print(name)
   
   with h5py.File(p) as f:
       img = np.array(f['X'])[...,0]
       points = []
       img_max = np.max(img)

       for i in range(1,33):
           treatments_out[name][i] = 0
           mask = f['y'][...,i]

           if mask.any() :
               tooth = np.where(mask, img, 0)
               middle = np.ceil([np.mean(np.nonzero(tooth)[1]), np.mean(np.nonzero(tooth)[0])])
               
               distrib =  np.vstack(np.unique(tooth, return_counts=True)).T

               gm = GaussianMixture(n_components=5).fit(distrib)
               to_write = str(i) + " : " + str(int(np.ceil(np.max(gm.means_[...,0]))))

               if np.any([gm.means_[...,0] >= thresh*img_max ]):
                   points.append([*middle, "green", to_write])
                   treatments_out[name][i] = 1
               else:
                   points.append([*middle, "red", to_write])
                   treatments_out[name][i] = 0

       img = np.repeat(img[..., np.newaxis], 3, axis=2)
       img = draw_annotation(img, points)
       plt.imsave("OUT/" + name + ".png", img.astype(np.uint8))

In [ ]:
with open('out.csv', 'w') as f:
    for key in sorted(treatments_out):
        f.write(key)
        for tooth in treatments_out[key]:
            f.write(", " + str(tooth) + " : " + str(treatments_out[key][tooth]))
        f.write("\n")